# GDMA Project
Author: Julian Schelb (1069967)

In [1]:
from neo4j import GraphDatabase
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

### Connection to the database instance

In [2]:
driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "subatomic-shrank-Respond"))
database_name = "cddb"
session = driver.session(database = database_name)

### Task 4: Searching and Ranking

Implement a simple search engine that enables search by artist, album and
song name/title. The results must be ranked based on importance. It is up to
you to come up with how the importance of each result is computed and you
must justify your decision (it goes without saying that you need to come up
with a meaningful definition). However, the importance should ideally take into
account user preferences/likes. As such, this task is split in two parts:

**1. Write a Cypher query that adds a relationship :LIKES between a node with**

label :User and an artist, album, or song. Every user should be identified
just by a numerical userID (no more information is necessary). If a user
already exists in the system, no additional node should be added. After
coming up with the necessary Cypher query, add a significant number of
users and likes.

**2. Implement a simple Python function that has the following arguments:**

- the userID of the user submitting the search (the user ID may not
exist in the database),
- a string that contains one or more keywords for the search, and
- an optional argument that indicates whether the search is on all or
a specific field, i.e., artist, album, song.
The search must return exactly 10 results.

Python must only be used to call the database. You should not write any
code in Python that implements functionality necessary for the task. However,
submitting multiple queries in the same function call is allowed. Also, for this
task of the project, you are not only allowed but also encouraged to use functions
from the GDS library of Neo4j. Hence, before making any decisions, have a
careful look at the available functions. Again, you have to justify the use of any
function that you employ

#### Adding Example Users

Function for creating a user node:

In [3]:
def createUser(driver, database_name: str = "cddb", user_id: int = None):
    
    if type(user_id) is not int:
        raise TypeError('User ID must be a Number')
    if user_id < 0:
        raise ValueError('User ID must be a positive Number')
        
        
    query = """
    MERGE (u:User {id:  $user_id})
    RETURN u.id as user_id
    """
    
    with driver.session(database = database_name) as session:
        session.run(query, user_id = user_id)
    

In [4]:
createUser(driver, user_id = 99)

Function for deleting a user node and all relations:

In [5]:
def deleteUser(driver, database_name: str = "cddb", user_id: int = None):
    
    if type(user_id) is not int:
        raise TypeError('User ID must be a Number')
    if user_id < 0:
        raise ValueError('User ID must be a positive Number')
        
        
    query = """
    MATCH (u:User)
    WHERE u.id = $user_id
    DETACH DELETE u
    """
    
    with driver.session(database = database_name) as session:
        session.run(query, user_id = user_id)

In [6]:
deleteUser(driver, user_id = 99)

Function for letting user like a song, album or artist:

In [7]:
def addLike(driver, database_name: str = "cddb", user_id: int = None, node_id: int = None, node_label: str = None):
    
    # Validate user_id
    if type(user_id) is not int:
        raise TypeError('User ID must be a Number')
    if user_id < 0:
        raise ValueError('User ID must be a positive Number')
    
    # Validate node_id
    if type(node_id) is not int:
        raise TypeError('Node ID must be a Number')
    if node_id < 0:
        raise ValueError('Node ID must be a positive Number')
        
    # Validate node_label    
    if node_label not in ["Song", "Album", "Artist"]:
        raise ValueError('Node type must be song, album or artist')
    
    createUser(driver, user_id = user_id)
    
    query = f"""
    MATCH (u:User)
    WHERE u.id = {user_id}
    MATCH (n: {node_label})
    WHERE n.id = {node_id}
    MERGE (u)-[r:LIKES]->(n)
    """
    
    with driver.session(database = database_name) as session:
        session.run(query)
    

In [8]:
addLike(driver, user_id = 99, node_id = 10, node_label = "Song")

Function for creating an example user who listens to a given genre:

In [19]:
def createExampleUser(driver, user_id: int = 1, genre: str = "rock", limit: int = 50):
    
    ####### CREATE USER #######
    
    deleteUser(driver, user_id = user_id)
    createUser(driver, user_id = user_id)
    
    ####### LIKE SONGS #######
    
    query = """
    MATCH (g:Genre)<-[r:BELONGS_TO]-(c:CD)
    MATCH (c)-[r2:CONTAINS]->(s:Song)
    WHERE g.genre = $genre
    WITH DISTINCT s 
    LIMIT $limit
    RETURN s.id as id
    """
    
    with driver.session(database = database_name) as session:
        results = session.run(query, user_id = user_id, genre = genre, limit = limit)
        for row in results:
            addLike(driver, user_id = user_id, node_id = row["id"], node_label = "Song")
            
    ####### LIKE ALBUMS #######
    
    query = """
    MATCH (g:Genre)<-[r:BELONGS_TO]-(c:CD)
    MATCH (c)-[r2:CONTAINS]->(s:Album)
    WHERE g.genre = $genre
    WITH DISTINCT s 
    LIMIT $limit
    RETURN s.id as id
    """
    
    with driver.session(database = database_name) as session:
        results = session.run(query, user_id = user_id, genre = genre, limit = limit)
        for row in results:
            addLike(driver, user_id = user_id, node_id = row["id"], node_label = "Album")
            
    ####### LIKE ARTISTS #######
    
    query = """
    MATCH (g:Genre)<-[r:BELONGS_TO]-(c:CD)
    MATCH (c)-[r2:CONTAINS]->(s:Artist)
    WHERE g.genre = $genre
    WITH DISTINCT s 
    LIMIT $limit
    RETURN s.id as id
    """
    
    with driver.session(database = database_name) as session:
        results = session.run(query, user_id = user_id, genre = genre, limit = limit)
        for row in results:
            addLike(driver, user_id = user_id, node_id = row["id"], node_label = "Artist")
    

In [20]:
createExampleUser(driver, user_id = 99, genre = "rock", limit = 50)

**Adding some Likes to model User Preference:**

In [24]:
# User 1 likes "Rock" music
createExampleUser(driver, user_id = 1, genre = "rock", limit = 50)
# User 2 likes "classic" music
createExampleUser(driver, user_id = 2, genre = "classic", limit = 50)
# User 3 likes "pop" music
createExampleUser(driver, user_id = 3, genre = "pop", limit = 50)
# User 4 likes "hip-hop" music
createExampleUser(driver, user_id = 4, genre = "hip-hop", limit = 50)
# User 5 likes "hip-hop" music
createExampleUser(driver, user_id = 5, genre = "hard rock", limit = 50)

***

**Implementation Explained:**

This search incorporates nodes corresponding to the input string on the one hand, and the previously liked albums, artists and songs on the other hand. Two separate scores are calculated in a similar manner. Separately calculated Centrality Scores form the basis. Separate graph projections are created for nodes that match the search input and for nodes that have been previously liked. Subsequently, a centrality score is calculated for each node in both. The basic idea is that the most relevant CDs will have a high centrality score. The "Content Match" score and the "User Preference" score are then combined in a weighted manner. The ten CDs with the highest combined score are presented to the user. 

![](Figures/search.png)

***

#### Demonstrating Search

In [1]:
from search import Search

**Create Instance of Search Engine:**

In [3]:
searchEngine = Search(url = "bolt://localhost:7687", user= "neo4j", 
                      password = "subatomic-shrank-Respond", database_name = "cddb")

**Search for Jimi Hendrix:**

In [4]:
user_id = 1
search_input = "Jimi Hendrix purple haze are you experienced"
search_mask = "all"

results = searchEngine.searchInGraph(user_id = 1, search_input = search_input, search_mask= search_mask)
results.head(30)

,nodeId,count,score_cont,score_pref,artists,albums,songs
0,7923,1,0.265149,0.000000,[signature licks],[jimi hendrix],"[fire (verse);, foxey lady (solo, slow);, fire..."
1,2023,2,0.133564,0.034758,[jimi hendrix],[experience hendrix - the best of jimi hendrix],"[bold as love, night bird flying, if 6 was 9, ..."
2,35734,2,0.133564,0.034758,[jimi hendrix],[are you experienced],"[i don't live today, 51st anniversary, stone f..."
3,677,2,0.133564,0.034758,[jimi hendrix],[are you experienced],"[third stone from the sun, remember, fire, can..."
4,46232,2,0.133564,0.034758,[jimi hendrix],[experience hendrix the best of jimi hendrix],"[foxey lady, dolly dagger, bold as love, if 6 ..."
5,136907,2,0.133564,0.034758,[jimi hendrix],[are you experienced],"[love or confusion, remember, highway chile, m..."
6,162186,2,0.133564,0.034758,[jimi hendrix],[experience hendrix: the best of jimi hendrix],"[if six was nine, foxey lady, bold as love, ni..."
7,20222,2,0.133564,0.017759,[jimi hendrix],[best of jimi hendrix],"[blues, blues, free spirit, star spangled bann..."
8,33321,2,0.133564,0.017759,[jimi hendrix],[astro man(alchemy); - studio outtakes 1966-68],"[purple haze 1 (4);, 51st anniversary (5);, la..."
9,30138,2,0.133564,0.017759,[jimi hendrix],[astro man box set],"[can you see me (pre unre);, purple haze (pre ..."


**Search for Beethoven:**

In [5]:
user_id = 2
search_input = "Ludwig van Beethoven Für Elise Symphony"
search_mask = "all"

results = searchEngine.searchInGraph(user_id = 1, search_input = search_input, search_mask= search_mask)
results.head(30)

,nodeId,count,score_cont,score_pref,artists,albums,songs
0,140152,1,0.144670,0.000000,[pollini maurizio],[kaiser 5.2],[erläuterungen von joachim kaiser mit musikbei...
1,33062,1,0.000000,0.255736,[ben harper],[fight for your mind],"[one road to freedom, give a man a home, oppre..."
2,167085,1,0.000000,0.238738,[jannedaarc],[arcadia],"[romancã, heavy damage, acid breath, dolls, wi..."
3,28698,1,0.000000,0.221739,[ben harper],[fight for your mind],"[oppression, excuse me mr, give a man a home, ..."
4,141988,1,0.108912,0.000000,[philip jones bläserensemble],[trumpet voluntary],"[sonata pian' e forte (giovanni gabrieli);, ea..."
5,166362,1,0.000000,0.187743,[the coral],[the invisible invasion],"[cripples crown, gina jones, leeslunchboxbyblu..."
6,13220,1,0.000000,0.170744,[groundhogs],[who will save the world],"[music is the food of thought, wages of peace,..."
7,11783,1,0.000000,0.170744,[groundhogs],[who will save the world],"[wages of peace, the grey maze, body in mind, ..."
8,124071,1,0.073155,0.000000,[ludwig van beethoven],[beethoven for meditation],"[piano and wind quintet andante cantabile, sep..."
9,114751,1,0.073155,0.000000,[ludwig van beethoven],[beethoven greatest hits],"[choral fantasy conclusion, moonlight sonata a..."
